In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [3]:
spark

In [17]:
#read dataset
df_pyspark=spark.read.csv('test1.csv',header=True,inferSchema=True)

In [18]:
df_pyspark.show()

+-------+----+----------+------+
|   Name| Age|Experience|Salary|
+-------+----+----------+------+
|    avi|  31|         4| 30000|
|  rahul|  30|         3| 27000|
|shubham|  33|         5| 50000|
| ganesh|  27|         3| 24000|
|sandesh|  24|         2| 20000|
| Harsha|  30|         4| 40000|
| mahesh|null|      null| 35000|
|   null|  34|        10| 38000|
|   null|  36|      null|  null|
+-------+----+----------+------+



In [19]:
training=df_pyspark.na.drop()

In [20]:
training.show()


+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|    avi| 31|         4| 30000|
|  rahul| 30|         3| 27000|
|shubham| 33|         5| 50000|
| ganesh| 27|         3| 24000|
|sandesh| 24|         2| 20000|
| Harsha| 30|         4| 40000|
+-------+---+----------+------+



In [21]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [22]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [23]:
[Age,Experience] ----> new feature ---> indepedent feature

SyntaxError: invalid syntax (<ipython-input-23-1c07cf03354b>, line 1)

In [24]:
from pyspark.ml.feature import VectorAssembler
featureassembler= VectorAssembler(inputCols=['Age','Experience'],outputCol="Independent Feature")

In [25]:
output = featureassembler.transform(training)

In [26]:
output.show()

+-------+---+----------+------+-------------------+
|   Name|Age|Experience|Salary|Independent Feature|
+-------+---+----------+------+-------------------+
|    avi| 31|         4| 30000|         [31.0,4.0]|
|  rahul| 30|         3| 27000|         [30.0,3.0]|
|shubham| 33|         5| 50000|         [33.0,5.0]|
| ganesh| 27|         3| 24000|         [27.0,3.0]|
|sandesh| 24|         2| 20000|         [24.0,2.0]|
| Harsha| 30|         4| 40000|         [30.0,4.0]|
+-------+---+----------+------+-------------------+



In [27]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Feature']

In [29]:
finalized_data=output.select('Independent Feature','Salary')

In [30]:
finalized_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|         [31.0,4.0]| 30000|
|         [30.0,3.0]| 27000|
|         [33.0,5.0]| 50000|
|         [27.0,3.0]| 24000|
|         [24.0,2.0]| 20000|
|         [30.0,4.0]| 40000|
+-------------------+------+



In [31]:
from pyspark.ml.regression import LinearRegression
train_data,test_data = finalized_data.randomSplit([.75,.25])

In [32]:
regressor = LinearRegression(featuresCol='Independent Feature',labelCol='Salary')

In [33]:
regressor=regressor.fit(train_data)

In [34]:
## coefficient
regressor.coefficients

DenseVector([800.0, 2200.0])

In [35]:
### Intercepts
regressor.intercept

-3599.999999999236

In [36]:
# Prediction
pred_result = regressor.evaluate(test_data)

In [37]:
pred_result.predictions.show()

+-------------------+------+-----------------+
|Independent Feature|Salary|       prediction|
+-------------------+------+-----------------+
|         [27.0,3.0]| 24000|24600.00000000006|
|         [30.0,4.0]| 40000|29200.00000000008|
|         [33.0,5.0]| 50000| 33800.0000000001|
+-------------------+------+-----------------+



In [38]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError

(9199.999999999958, 126479999.99999833)

In [3]:
spark.stop()